In [1]:
import simulation_model as sim

In [2]:
capacity_initial = {'housing' : 40, 'shelter' : 15}
housing_build_frequency = 63/365 # in years
end_of_simulation = 6 # in years
service_mean = {'housing' : (1/52)*(0+300+400)/3, 'shelter' : 0.0} # in years
arrival_rates = [35.0400, 42.0048, 46.2528, 46.2528, 41.6100, 37.4052] # expected number per year
build_rate_change_frequency = 1 # in years
build_rates = {'housing' : [3,6,7,10,8,4], 'shelter' : [2,2,0,-2,-1,-1]}
build_rates = {'housing' : [18, 36, 42, 60.1, 48, 24.9], 'shelter' : [12.256, 12, 0.2, -12.5, -6, -6]}
current_demand = 120
number_reps = 10
seed = 10

In [3]:
output = sim.simulate(end_of_simulation, number_reps, housing_build_frequency, build_rate_change_frequency, capacity_initial, service_mean, arrival_rates, build_rates, current_demand, seed)

[<simulation_model.Accommodation object at 0x7f3a6e4c6dd0>, <simulation_model.Accommodation object at 0x7f3a6e482690>, <simulation_model.Accommodation object at 0x7f3a6e61f910>, <simulation_model.Accommodation object at 0x7f3a6e4d5e90>, <simulation_model.Accommodation object at 0x7f3a6e4d6150>, <simulation_model.Accommodation object at 0x7f3a6e4d5cd0>, <simulation_model.Accommodation object at 0x7f3a6e4e9390>, <simulation_model.Accommodation object at 0x7f3a6e4e97d0>, <simulation_model.Accommodation object at 0x7f3a6e4e98d0>, <simulation_model.Accommodation object at 0x7f3a6e4d6010>, <simulation_model.Accommodation object at 0x7f3a6e4e9910>, <simulation_model.Accommodation object at 0x7f3a6e4e9890>, <simulation_model.Accommodation object at 0x7f3a6e4e9950>, <simulation_model.Accommodation object at 0x7f3a6e4e9990>, <simulation_model.Accommodation object at 0x7f3a6e4e99d0>, <simulation_model.Accommodation object at 0x7f3a6e4e9a10>, <simulation_model.Accommodation object at 0x7f3a6e4e9a5

customer 1083 leaves shelter at time 1.3808219178082195
customer 1083 enters housing at time 1.3808219178082195
customer 1100 enters shelter at time 1.3808219178082195
customer 1100 looks for housing at time 1.3808219178082195
customer 1190 arrives at time t = 1.388411453843286
current queue: 106
customer 1190 looks for shelter at time 1.388411453843286
get queue is 106<class 'list'>
customer 1191 arrives at time t = 1.4351506867796868
current queue: 107
customer 1191 looks for shelter at time 1.4351506867796868
get queue is 107<class 'list'>
customer 1192 arrives at time t = 1.4375482035766936
current queue: 108
customer 1192 looks for shelter at time 1.4375482035766936
get queue is 108<class 'list'>
customer 1032 leaves housing at time 1.4405022926208422
customer 1084 leaves shelter at time 1.4405022926208422
customer 1084 enters housing at time 1.4405022926208422
customer 1101 enters shelter at time 1.4405022926208422
customer 1101 looks for housing at time 1.4405022926208422
custom

customer 2325 arrives at time t = 3.6412877321158352
current queue: 136
customer 2325 looks for shelter at time 3.6412877321158352
get queue is 136<class 'list'>
customer 2326 arrives at time t = 3.6482066414169667
current queue: 137
customer 2326 looks for shelter at time 3.6482066414169667
get queue is 137<class 'list'>
customer 2327 arrives at time t = 3.657588833283957
current queue: 138
customer 2327 looks for shelter at time 3.657588833283957
get queue is 138<class 'list'>
customer 2328 arrives at time t = 3.6766123036905243
current queue: 139
customer 2328 looks for shelter at time 3.6766123036905243
get queue is 139<class 'list'>
customer 2329 arrives at time t = 3.7145612501272285
current queue: 140
customer 2329 looks for shelter at time 3.7145612501272285
get queue is 140<class 'list'>
customer 2097 leaves housing at time 3.723921438913754
customer 2190 leaves shelter at time 3.723921438913754
customer 2190 enters housing at time 3.723921438913754
customer 2209 enters shelte

customer 3221 enters shelter at time 2.9342465753424665
customer 3221 looks for housing at time 2.9342465753424665
customer 3345 arrives at time t = 2.95138951733711
current queue: 142
customer 3345 looks for shelter at time 2.95138951733711
get queue is 142<class 'list'>
customer 3346 arrives at time t = 2.959242623208111
current queue: 143
customer 3346 looks for shelter at time 2.959242623208111
get queue is 143<class 'list'>
customer 3347 arrives at time t = 2.9658531930570753
current queue: 144
customer 3347 looks for shelter at time 2.9658531930570753
get queue is 144<class 'list'>
customer 3348 arrives at time t = 2.9977667530371805
current queue: 145
customer 3348 looks for shelter at time 2.9977667530371805
get queue is 145<class 'list'>
customer 3349 arrives at time t = 3.006784585329863
current queue: 146
customer 3349 looks for shelter at time 3.006784585329863
get queue is 146<class 'list'>
customer 3350 arrives at time t = 3.017049961715443
current queue: 147
customer 335

In [4]:
print(output)

[[  0   0   0   0   0   0   0   0   0   0]
 [ 70  66  69  71  77  66  65  71  67  64]
 [ 71  75  73  71  85  71  71  72  69  68]
 [ 78  78  74  75  84  74  72  75  73  67]
 [ 79  83  76  82  85  75  71  77  79  70]
 [ 82  86  78  88  87  80  71  81  80  75]
 [ 85  85  79  89  86  82  72  86  86  76]
 [ 82  90  84  89  87  88  82  90  89  75]
 [ 82  97  90  90  89  98  84  97  87  78]
 [ 80  99  99  96  94  99  86 103  89  87]
 [ 84  99 102  97  97 103  91 108  89  95]
 [ 90 105 106 103 100 106  94 105  93 102]
 [ 93 111 110 108 107 107  95 110  89 102]
 [ 92 118 108 111 108 116 101 110  92 101]
 [ 96 120 110 118 113 121 106 115  98 106]
 [ 97 128 115 126 119 124 112 119 102 116]
 [104 130 120 134 123 130 111 125 104 120]
 [109 133 127 144 126 134 115 133 112 124]
 [110 138 131 144 132 141 113 142 121 132]
 [118 139 137 147 134 145 117 144 121 137]
 [125 141 142 150 141 144 122 152 130 140]
 [129 149 147 153 144 147 118 155 135 148]
 [130 154 150 154 150 146 124 163 139 153]
 [135 157 1